In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import sqlalchemy
import os

In [2]:
conn = psycopg2.connect(
    user="dcat", 
    password="dcat", 
    host="127.0.0.1", 
    port="54320", 
    database="dcat")

In [ ]:
# vault login -address="https -method=oidc
# vault read --address="https://vault.adeo.no" postgresql/preprod-fss/creds/datacatalog-term-admin

conn = psycopg2.connect(
    user="v-oidc-Pau-data-cat-JttunzZohPHCVqCALH7e-1574349384", 
    password="A1a-KVbk7te3wn4ndtue", 
    host="A01DBVL010.adeo.no", 
    port="5432", 
    database="data-catalog-metadata")

In [3]:
cur = conn.cursor()

In [4]:
print ( conn.get_dsn_parameters(),"\n")

{'user': 'dcat', 'dbname': 'dcat', 'host': '127.0.0.1', 'port': '54320', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [5]:
cur.execute("SELECT version();")
record = cur.fetchone()
print("You are connected to - ", record,"\n")

You are connected to -  ('PostgreSQL 12.1 (Debian 12.1-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',) 



In [6]:
cur.execute("CREATE EXTENSION IF NOT EXISTS pgcrypto;")

In [7]:
conn.commit()

In [8]:
cur.execute("drop index if exists idx_tbl_edge_n1")
cur.execute("drop index if exists idx_tbl_edge_n2")
cur.execute("drop table if exists tbl_edge")

# Node table

In [9]:
cur.execute("drop table if exists tbl_node")
cur.execute("""
create table tbl_node 
(  
  id SERIAL PRIMARY KEY,
  guid UUID default gen_random_uuid(),
  prop JSONB, 
  created TIMESTAMP default now() 
);  
""")

In [10]:
cur.execute("drop table if exists tbl_node")
cur.execute("""
create table tbl_node 
(  
  id SERIAL PRIMARY KEY,
  prop JSONB, 
  created TIMESTAMP default now() 
);  
""")

In [11]:
sql = "select * from tbl_node"
df = sqlio.read_sql_query(sql, conn)
df.head()

,id,prop,created


In [12]:
conn.commit()
sql = 'select * from pg_catalog.pg_user'
df_users = sqlio.read_sql_query(sql, conn)

# Edges 

In [13]:
cur.execute("drop table if exists tbl_edge")
cur.execute("""
create table tbl_edge     -- What is relationship between ID1 to ID2      
(  
  n1 int references tbl_node(id),    
  n2 int references tbl_node(id),
  prop JSONB,
  created timestamp default now(),  
  check (n1<>n2),  
  unique (n1,n2)  
);  
""")

In [14]:
sql = "select * from tbl_edge"
df = sqlio.read_sql_query(sql, conn)
df.head()

,n1,n2,prop,created


In [15]:
cur.execute("drop index if exists idx_tbl_edge_n1")
cur.execute("create index idx_tbl_edge_n1 on tbl_edge(n1)")

In [16]:
cur.execute("drop index if exists idx_tbl_edge_n2")
cur.execute("create index idx_tbl_edge_n2 on tbl_edge(n2)")

In [17]:
conn.commit()